# Build a Multi-User Conversational Tool-Calling Agentic AI Research Assistant with LangChain

This demo will cover building AI Agents with the legacy LangChain `AgentExecutor`. These are fine for getting started, but for working with more advanced agents, LangChain recommends to use LangGraph, which we will cover in the future demos.

Agents are systems that use an LLM as a reasoning engine to determine which actions to take and what the inputs to those actions should be. The results of those actions can then be fed back into the agent and it determines whether more actions are needed, or whether it is okay to stop.

Here we will build a multi-user Conversational Agent which can refer to previous conversations and give more contextual answers by storing agent messages to a SQL database

![](https://i.imgur.com/lHWqaT9.png)



## Install OpenAI, and LangChain dependencies

In [1]:
# !pip install langchain==0.3.14
# !pip install langchain-openai==0.3.0
# !pip install langchain-community==0.3.14

In [2]:
# !pip install markitdown

## Enter Open AI API Key

In [3]:
# from getpass import getpass

# OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Enter Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

In [4]:
# TAVILY_API_KEY = getpass('Enter Tavily Search API Key: ')

## Enter WeatherAPI API Key

Get a free API key from [here](https://www.weatherapi.com/signup.aspx)

In [5]:
# WEATHER_API_KEY = getpass('Enter WeatherAPI API Key: ')

## Setup Environment Variables

In [6]:
# import os

# os.environ['OPENAI_API_KEY'] = OPENAI_KEY
# os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [7]:
# =============================================================================
# LOAD ENVIRONMENT VARIABLES
# =============================================================================
# This loads API keys from a .env file in the project directory.
# Your .env file should contain:
#   OPENAI_API_KEY=your_openai_key
#   TAVILY_API_KEY=your_tavily_key  
#   WEATHER_API_KEY=your_openweathermap_key
# =============================================================================

from dotenv import load_dotenv, find_dotenv

# find_dotenv() searches for .env file in current and parent directories
load_dotenv(find_dotenv())

True

## Create Tools

Here we create two custom tools which are wrappers on top of the [Tavily API](https://tavily.com/#api) and [WeatherAPI](https://www.weatherapi.com/)

- Web Search tool with information extraction
- Weather tool

![](https://i.imgur.com/TyPAYXE.png)

In [ ]:
from langchain_core.tools import tool
from markitdown import MarkItDown
from langchain_community.tools.tavily_search import TavilySearchResults
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, TimeoutError
import requests
import json
from warnings import filterwarnings
from IPython.display import display, Markdown

filterwarnings('ignore')

tavily_tool = TavilySearchResults(max_results=5,
                                  search_depth='advanced',
                                  include_answer=False,
                                  include_raw_content=True)
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br"
})
md = MarkItDown(requests_session=session)

@tool
def search_web_extract_info(query: str) -> list:
    """Search the web for a query and extracts useful information from the search links."""
    print('Calling web search tool')
    results = tavily_tool.invoke(query)
    docs = []

    def extract_content(url):
        """Helper function to extract content from a URL."""
        extracted_info = md.convert(url)
        text_title = extracted_info.title.strip()
        text_content = extracted_info.text_content.strip()
        return text_title + '\n' + text_content

    with ThreadPoolExecutor() as executor:
        for result in tqdm(results):
            try:
                future = executor.submit(extract_content, result['url'])
                # Wait for up to 60 seconds for the task to complete
                content = future.result(timeout=60)
                docs.append(content)
            except TimeoutError:
                print(f"Extraction timed out for url: {result['url']}")
            except Exception as e:
                print(f"Error extracting from url: {result['url']} - {e}")

    return docs


# =============================================================================
# TOOL 2: Weather Information Tool  
# =============================================================================
# This tool fetches real-time weather data using OpenWeatherMap API

@tool
def get_weather(query: str) -> dict:
    """
    Fetch current weather data for a specified location using OpenWeatherMap API.
    
    Args:
        query (str): The city name to get weather information for
        
    Returns:
        dict: Weather data including temperature, humidity, wind speed, etc.
              Returns "Weather Data Not Found" if the location is invalid
    """
    # Construct API URL with city name and API key
    # Note: The ',IN' suffix restricts search to India - modify as needed
    url = f"https://api.openweathermap.org/data/2.5/weather?q={query},IN&appid={os.getenv('WEATHER_API_KEY')}&units=metric"

    response = requests.get(url)
    data = response.json()
    
    # Validate response by checking if 'name' field exists
    if data.get("name"):
        return data
    else:
        return "Weather Data Not Found"

## Build and Test AI Agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using a tool calling agent to bind the tools to the agent with a prompt. We will also add in the capability to store historical conversations as memory

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYS_PROMPT = """Act as a helpful assistant.
                You run in a loop of Thought, Action, PAUSE, Observation.
                At the end of the loop, you output an Answer.
                Use Thought to describe your thoughts about the question you have been asked.
                Use Action to run one of the actions available to you - then return PAUSE.
                Observation will be the result of running those actions.
                Repeat till you get to the answer for the given user query.

                Use the following workflow format:
                  Question: the input task you must solve
                  Thought: you should always think about what to do
                  Action: the action to take which can be any of the following:
                            - break it into smaller steps if needed
                            - see if you can answer the given task with your trained knowledge
                            - call the most relevant tools at your disposal mentioned below in case you need more information
                  Action Input: the input to the action
                  Observation: the result of the action
                  ... (this Thought/Action/Action Input/Observation can repeat N times)
                  Thought: I now know the final answer
                  Final Answer: the final answer to the original input question

                Tools at your disposal to perform tasks as needed:
                  - get_weather: whenever user asks get the weather of a place.
                  - search_web_extract_info: whenever user asks for specific information or if you don't know the answer.
             """

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history", optional=True),
        ("human", "{query}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="Act as a helpful assistant.\n                You run in a loop of Thought, Action, PAUSE, Observation.\n                At the end of the loop, you output an Answer.\n                Use Thought to describe your thoughts about the question you have been asked.\n                Use Action to run one of the actions available to you - then return PAUSE.\n                Observation will be the result of running those actions.\n                Repeat till you get to the answer for the given user query.\n\n                Use the following workflow format:\n                  Question: the input task you must solve\n                  Thought: you should always think about what to do\n                  Action: the action to take which can be any of the following:\n                            - break it into smaller steps if needed\n                            - see if you can

Now, we can initalize the agent with the LLM, the prompt, and the tools.

The agent is responsible for taking in input and deciding what actions to take.

REMEMBER the Agent does not execute those actions - that is done by the AgentExecutor

Note that we are passing in the model `chatgpt`, not `chatgpt_with_tools`.

That is because `create_tool_calling_agent` will call `.bind_tools` for us under the hood.

This should ideally be used with an LLM which supports tool \ function calling

In [ ]:
from langchain.agents import create_tool_calling_agent
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [search_web_extract_info, get_weather]
agent = =-
(chatgpt, tools, prompt_template)

Finally, we combine the `agent` (the brains) with the `tools` inside the `AgentExecutor` (which will repeatedly call the agent and execute tools).

In [11]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               early_stopping_method='force',
                               max_iterations=10)

## Build and Test Multi-User Conversational AI Agent

We will now use `SQLChatMessageHistory` which we learnt in the previous module to store separate conversation histories per user or session.

This will help us build a conversational Agentic Chatbot which will be accessed by many users at the same time

In [18]:
# removes the memory database file - usually not needed
# you can run this only when you want to remove ALL conversation histories
# ok if you get rm: cannot remove 'memory.db': No such file or directory  because initially no memory exists
!rm memory.db

rm: memory.db: No such file or directory


In [19]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# used to retrieve conversation history from database
# based on a specific user or session ID
def get_session_history_db(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

# create a conversation chain + agent which can load memory based on specific user or session id
agentic_chatbot = RunnableWithMessageHistory(
    agent_executor,
    get_session_history_db,
    input_messages_key="query",
    history_messages_key="history",
)

# function to call the agent show results per user session
from IPython.display import display, Markdown
def chat_with_agent(prompt: str, session_id: str):
    response = agentic_chatbot.invoke({"query": prompt},
                                      {'configurable': { 'session_id': session_id}})
    display(Markdown(response['output']))

Let's now simulate User 1 using the agent

In [20]:
user_id = 'john001'
prompt = "Summarize the key points discussed in Nvidia's Q4 2024 earnings call"
chat_with_agent(prompt, user_id)

/Users/sourav.banerjee/Documents/Codebases/2. AI ENGINEERING/LangChain_Demystified/.venv/lib/python3.13/site-packages/langchain_core/runnables/history.py:606: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(


Calling web search tool


 20%|██        | 1/5 [00:00<00:00,  9.77it/s]Cannot set gray non-stroke color because /'P25' is an invalid float value
Cannot set gray non-stroke color because /'P46' is an invalid float value
Cannot set gray non-stroke color because /'P137' is an invalid float value
Cannot set gray non-stroke color because /'P172' is an invalid float value
Cannot set gray non-stroke color because /'P205' is an invalid float value
Cannot set gray non-stroke color because /'P238' is an invalid float value
Cannot set gray non-stroke color because /'P271' is an invalid float value
Cannot set gray non-stroke color because /'P306' is an invalid float value
Cannot set gray non-stroke color because /'P339' is an invalid float value
Cannot set gray non-stroke color because /'P374' is an invalid float value
Cannot set gray non-stroke color because /'P442' is an invalid float value
 40%|████      | 2/5 [00:00<00:01,  1.93it/s]

Error extracting from url: https://s201.q4cdn.com/141608511/files/doc_financials/2024/q4/NVDA-F4Q24-Quarterly-Presentation-FINAL.pdf - 'NoneType' object has no attribute 'strip'


 60%|██████    | 3/5 [00:01<00:01,  1.78it/s]

Error extracting from url: https://www.youtube.com/watch?v=hV6WxM3S80g - 'NoneType' object has no attribute 'strip'


100%|██████████| 5/5 [00:03<00:00,  1.55it/s]

Error extracting from url: https://nvidianews.nvidia.com/_gallery/download_pdf/65d669a33d63329bbf62672a/ - 'NoneType' object has no attribute 'strip'


Nvidia's Q4 2024 earnings call highlighted several key points:

1. **Financial Performance**: Nvidia reported a significant increase in revenue, reaching $22.1 billion for the quarter, a 22% increase from the previous quarter and a 265% rise from the same period a year ago. The fiscal year 2024's revenue totaled $60.9 billion, up by 126%. The earnings per share (EPS) also saw substantial growth, with GAAP EPS at $4.93 and Non-GAAP EPS at $5.16.

2. **Data Center and AI Growth**: Approximately 40% of Nvidia's record data center revenue of $18.4 billion was attributed to AI inference. The company's AI enterprise software reached an annualized revenue run rate of $1 billion, indicating potential growth in leveraging AI across various industries.

3. **Market Sentiment and Stock Performance**: Following the earnings announcement, Nvidia's stock surged by more than 8% in after-hours trading, reflecting positive market sentiment and investor confidence in the company's growth trajectory.

4. **Strategic Initiatives and Challenges**: Nvidia's strategic focus on AI and accelerated computing, along with its expansive ecosystem of partnerships, positions it well for sustained growth. Despite facing regulatory challenges in China, Nvidia adapted by offering alternative products, mitigating potential impacts on its business.

Overall, Nvidia's Q4 2024 earnings call underscored its strong financial performance, strategic focus on AI, and ability to navigate regulatory challenges, positioning it for continued growth and innovation.

In [21]:
prompt = "What about Intel?"
chat_with_agent(prompt, user_id)

Calling web search tool


 40%|████      | 2/5 [00:01<00:02,  1.40it/s]

Error extracting from url: https://morethanmoore.substack.com/p/intel-2024-q4-financials - 'NoneType' object has no attribute 'strip'


 60%|██████    | 3/5 [00:02<00:01,  1.05it/s]

Error extracting from url: https://www.fool.com/earnings/call-transcripts/2025/01/30/intel-intc-q4-2024-earnings-call-transcript/ - 'NoneType' object has no attribute 'strip'


100%|██████████| 5/5 [00:05<00:00,  1.17s/it]

Error extracting from url: https://newsroom.intel.com/corporate/intel-reports-fourth-quarter-full-year-2024-financial-results - 'NoneType' object has no attribute 'strip'


Intel's Q4 2024 earnings call highlighted several key points:

1. **Financial Performance**: Intel reported Q4 revenue of $14.3 billion, which was a 7% sequential increase and surpassed guidance. The non-GAAP gross margin was 42.1%, exceeding expectations by 260 basis points. Earnings per share (EPS) were $0.13, slightly above the guidance of $0.12. However, the full-year 2024 revenue was $53.1 billion, down 2.1% year-over-year, with a full-year EPS of minus $0.13.

2. **Challenges and Guidance**: Despite surpassing guidance, Intel faces challenges such as increased competition in the AI PC category and a lack of significant participation in the cloud-based AI data center market. The company reported a significant operating loss in its foundry services. For Q1 2025, Intel provided a revenue guidance of $11.7 billion to $12.7 billion, with a breakeven EPS on a non-GAAP basis.

3. **Strategic Initiatives**: Intel is focusing on its AI PC CPUs and plans to ship over 100 million systems by the end of 2025. The launch of Panther Lake on Intel 18A is on track for the second half of 2025. Intel Foundry is making progress with Intel 18A, showing competitive offerings and a healthy pipeline of potential customers.

4. **Leadership Changes**: The company appointed two interim co-CEOs, David Zinsner and Michelle Johnston Holthaus, following the departure of CEO Pat Gelsinger. The search for a new CEO is ongoing.

5. **Market Dynamics**: Intel's guidance reflects macroeconomic uncertainty and seasonality, impacting all product segments. The company is taking a conservative approach to capital deployment and focusing on building a world-class foundry.

Overall, Intel's Q4 2024 earnings call highlighted its ability to surpass guidance amidst market challenges, while also outlining strategic initiatives and leadership changes to enhance its competitive position.

In [22]:
prompt = "Which company seems to be doing better?"
chat_with_agent(prompt, user_id)

Based on the Q4 2024 earnings calls for both Nvidia and Intel, Nvidia appears to be performing better overall. Here are some key points that support this assessment:

1. **Revenue Growth**: Nvidia reported a significant increase in revenue, with a 22% sequential increase and a 265% year-over-year rise, reaching $22.1 billion for the quarter. In contrast, Intel's revenue for the quarter was $14.3 billion, with a 7% sequential increase but a decline in full-year revenue by 2.1% year-over-year.

2. **Profitability**: Nvidia's earnings per share (EPS) showed substantial growth, with both GAAP and Non-GAAP EPS significantly higher than the previous year. Intel, on the other hand, reported a full-year EPS of minus $0.13, indicating a loss.

3. **Market Sentiment**: Nvidia's stock surged by more than 8% in after-hours trading following the earnings announcement, reflecting positive market sentiment. Intel's guidance reflects macroeconomic uncertainty, and the company is taking a conservative approach.

4. **Strategic Positioning**: Nvidia's strong focus on AI and data center growth, with a significant portion of revenue coming from AI inference, positions it well for future growth. Intel is facing challenges in the AI PC category and cloud-based AI data center market, although it is making strategic moves in its foundry services.

5. **Leadership Stability**: Nvidia did not report any major leadership changes, while Intel is undergoing a leadership transition with the appointment of interim co-CEOs and the search for a new CEO.

Overall, Nvidia's strong financial performance, strategic focus on AI, and positive market sentiment suggest it is currently in a better position compared to Intel, which is facing challenges and undergoing leadership changes.

Let's now simulate User 2 using the agent

In [23]:
user_id = 'bond007'
prompt = "how is the weather in Bangalore today? Show detailed statistics"
chat_with_agent(prompt, user_id)

The weather in Bangalore today is as follows:

- **Condition**: Mist
- **Temperature**: 14.84°C
- **Feels Like**: 14.76°C
- **Minimum Temperature**: 13.9°C
- **Maximum Temperature**: 15.28°C
- **Pressure**: 1015 hPa
- **Humidity**: 91%
- **Visibility**: 1800 meters
- **Wind Speed**: 5.36 m/s
- **Wind Direction**: 74° (East-Northeast)
- **Wind Gust**: 12.96 m/s
- **Cloudiness**: 20%

The sun will rise at 5:43 AM and set at 5:51 PM local time.

In [26]:
user_id = 'bond007'
prompt = "what about Mumbai?"
chat_with_agent(prompt, user_id)

The weather in Mumbai today is as follows:

- **Condition**: Smoke
- **Temperature**: 22.99°C
- **Feels Like**: 22.81°C
- **Minimum Temperature**: 22.99°C
- **Maximum Temperature**: 22.99°C
- **Pressure**: 1012 hPa
- **Humidity**: 56%
- **Visibility**: 3000 meters
- **Wind Speed**: 2.57 m/s
- **Wind Direction**: 100° (East)
- **Cloudiness**: 18%

The sun will rise at 6:13 AM and set at 6:53 PM local time.

Comparing the temperatures, Mumbai is currently warmer than Bangalore, which has a temperature of 14.84°C.

In [27]:
user_id = 'bond007'
prompt = "which city is hotter?"
chat_with_agent(prompt, user_id)

Based on the current weather data:

- **Bangalore**: 14.84°C
- **Mumbai**: 22.99°C

Mumbai is currently hotter than Bangalore.